In [2]:
import requests
import pandas as pd
import json

In [54]:
# sql_query = f'SELECT "OBSERVATORY_NAME", AVG(CAST("AVERAGE_HUMIDITY" AS numeric)) AS AVERAGE_HUMIDITY FROM "TABLE_PLACEHOLDER" GROUP BY "OBSERVATORY_NAME"'
sql_query = '''
WITH day_table AS (
    SELECT EXTRACT(day FROM CAST("DATE_TIME" AS date)) AS "DAY", * 
    FROM "TABLE_PLACEHOLDER"
    LIMIT 100
)

SELECT "DAY", AVG(CAST("AVERAGE_HUMIDITY" AS numeric))::numeric(5,2) AS average_humidity
FROM day_table
GROUP BY "DAY"
ORDER BY "DAY"
'''

In [80]:
sql_query = '''
SELECT * FROM "TABLE_PLACEHOLDER" LIMIT 10
'''

In [81]:
def run_sql(sql_query, table, fields=False):
    sql_query = sql_query.replace('TABLE_PLACEHOLDER',table)
    source = requests.get(f'https://data.ibb.gov.tr/api/3/action/datastore_search_sql?sql={sql_query}')
    try:
        result = pd.DataFrame(json.loads(source.text)['result']['records'])
        if fields:
            return pd.DataFrame(json.loads(source.text)['result']['fields'])
    except KeyError:
        result = json.loads(source.text)
    return result

In [88]:
columns = run_sql(sql_query, "8ca0081a-333c-4e86-8ea2-ab8741bbfc95", True)['id'].tolist()

In [130]:
print(columns)

['_id', '_full_text', 'DATE_TIME', 'OBSERVATORY_NAME', 'SENSOR_TYPE', 'MINIMUM_TEMPERATURE', 'MAXIMUM_TEMPERATURE', 'AVERAGE_TEMPERATURE', 'MINIMUM_HUMIDITY', 'MAXIMUM_HUMIDITY', 'AVERAGE_HUMIDITY', 'MINIMUM_WIND', 'MAXIMUM_WIND', 'AVERAGE_WIND', 'MINIMUM_DIRECTIONOFWIND', 'MAXIMUM_DIRECTIONOFWIND', 'AVERAGE_DIRECTIONOFWIND', 'MINIMUM_PRECIPITATION', 'MAXIMUM_PRECIPITATION', 'AVERAGE_PRECIPITATION', 'MINIMUM_ROAD_TEMPERATURE', 'MAXIMUM_ROAD_TEMPERATURE', 'AVERAGE_ROAD_TEMPERATURE', 'MINIMUM_FELT_TEMPERATURE', 'MAXIMUM_FELT_TEMPERATURE', 'AVERAGE_FELT_TEMPERATURE']


In [109]:
col_string ='CAST ("' + '" AS numeric(5,2)), CAST ("'.join(columns[5:]) + '" AS numeric(5,2))'
print(col_string)

CAST ("MINIMUM_TEMPERATURE" AS numeric(5,2)), CAST ("MAXIMUM_TEMPERATURE" AS numeric(5,2)), CAST ("AVERAGE_TEMPERATURE" AS numeric(5,2)), CAST ("MINIMUM_HUMIDITY" AS numeric(5,2)), CAST ("MAXIMUM_HUMIDITY" AS numeric(5,2)), CAST ("AVERAGE_HUMIDITY" AS numeric(5,2)), CAST ("MINIMUM_WIND" AS numeric(5,2)), CAST ("MAXIMUM_WIND" AS numeric(5,2)), CAST ("AVERAGE_WIND" AS numeric(5,2)), CAST ("MINIMUM_DIRECTIONOFWIND" AS numeric(5,2)), CAST ("MAXIMUM_DIRECTIONOFWIND" AS numeric(5,2)), CAST ("AVERAGE_DIRECTIONOFWIND" AS numeric(5,2)), CAST ("MINIMUM_PRECIPITATION" AS numeric(5,2)), CAST ("MAXIMUM_PRECIPITATION" AS numeric(5,2)), CAST ("AVERAGE_PRECIPITATION" AS numeric(5,2)), CAST ("MINIMUM_ROAD_TEMPERATURE" AS numeric(5,2)), CAST ("MAXIMUM_ROAD_TEMPERATURE" AS numeric(5,2)), CAST ("AVERAGE_ROAD_TEMPERATURE" AS numeric(5,2)), CAST ("MINIMUM_FELT_TEMPERATURE" AS numeric(5,2)), CAST ("MAXIMUM_FELT_TEMPERATURE" AS numeric(5,2)), CAST ("AVERAGE_FELT_TEMPERATURE" AS numeric(5,2))


In [189]:
avg_colstring = ''
for col in columns[5:]:
    avg_colstring+= f'AVG("{col}")::numeric(5,2) AS "{col}",'
avg_colstring = avg_colstring[:-1]

In [190]:
sql_query = f'''
WITH temp_table AS (
SELECT {col_string}, DATE_TRUNC('day', "DATE_TIME"::date)::date AS day
FROM "TABLE_PLACEHOLDER" 
LIMIT 100
)

SELECT {avg_colstring}, "day"
FROM temp_table
GROUP BY "day"
ORDER BY "day"
'''

In [191]:
run_sql(sql_query, "8ca0081a-333c-4e86-8ea2-ab8741bbfc95")

,AVERAGE_FELT_TEMPERATURE,AVERAGE_ROAD_TEMPERATURE,MINIMUM_FELT_TEMPERATURE,MINIMUM_PRECIPITATION,MAXIMUM_HUMIDITY,MAXIMUM_FELT_TEMPERATURE,MAXIMUM_PRECIPITATION,MAXIMUM_TEMPERATURE,AVERAGE_HUMIDITY,AVERAGE_WIND,...,AVERAGE_TEMPERATURE,MAXIMUM_ROAD_TEMPERATURE,MINIMUM_HUMIDITY,day,MINIMUM_DIRECTIONOFWIND,MINIMUM_WIND,MAXIMUM_DIRECTIONOFWIND,MINIMUM_ROAD_TEMPERATURE,MAXIMUM_WIND,AVERAGE_PRECIPITATION
0,4.90,-63.94,3.80,0.00,90.61,6.75,0.00,5.71,87.90,2.08,...,5.48,-63.63,86.44,2020-01-01,138.33,0.68,330.67,-64.17,4.46,0.00
1,4.70,2.46,2.92,0.00,95.20,5.78,0.00,3.46,93.83,0.34,...,3.00,2.70,92.40,2020-01-02,17.00,0.00,349.00,2.30,0.73,0.00
2,5.55,5.25,4.36,0.00,74.50,6.39,0.00,5.69,71.55,1.58,...,5.55,5.43,69.13,2020-01-04,56.00,1.05,311.00,5.05,2.39,0.00
3,3.36,-46.65,2.68,0.34,97.30,4.07,0.62,5.27,97.03,3.81,...,5.21,-46.60,96.80,2020-01-05,16.00,2.06,193.00,-46.65,5.66,0.46
4,4.91,-99.00,3.80,-99.00,91.00,5.93,-99.00,7.90,89.90,6.50,...,7.69,-99.00,89.00,2020-01-06,11.00,4.40,89.00,-99.00,10.70,-99.00
5,1.27,-47.45,0.12,-49.50,90.05,2.69,-49.50,5.12,89.25,8.74,...,5.01,-47.40,88.25,2020-01-07,4.00,5.27,202.50,-47.45,12.87,-49.50
6,3.05,-73.00,1.49,0.00,85.48,4.64,0.03,4.96,82.65,3.64,...,4.74,-72.99,79.75,2020-01-08,1.88,1.54,357.38,-73.04,6.53,0.00
7,4.61,-99.00,3.28,-33.00,84.00,6.40,-33.00,5.33,83.26,1.60,...,4.88,-99.00,82.33,2020-01-09,22.67,0.60,313.00,-99.00,2.80,-33.00
8,3.93,-73.62,2.61,0.00,94.40,6.30,0.00,3.84,93.14,0.81,...,3.29,-73.55,91.10,2020-01-10,65.50,0.32,255.00,-73.70,1.34,0.00
9,9.63,-99.00,8.63,0.00,70.50,11.07,0.00,9.55,67.70,1.43,...,9.13,-99.00,64.00,2020-01-11,74.50,0.40,306.50,-99.00,2.55,0.00


In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time

In [ ]:
source = requests.get()